# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
from sqlalchemy import create_engine 
import numpy as np
import pandas as pd 
import string 
import re 
import nltk
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import FunctionTransformer
import pickle 

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet') # download for lemmatization

In [ ]:
# In order to process the data we adapt code from the following web site: 
# https://blog.chapagain.com.np/python-nltk-twitter-sentiment-analysis-natural-language-processing-nlp/


from nltk.corpus import stopwords 
stopwords_english = stopwords.words('english')
 
from nltk.stem.porter import PorterStemmer


from nltk.stem.wordnet import WordNetLemmatizer


from nltk.tokenize import TweetTokenizer


# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

In [ ]:
# good resource 
# https://stackoverflow.com/questions/20209600/pandas-dataframe-remove-constant-column

def drop_constant_columns(df):
    """
    Drops constant value columns of pandas dataframe.
    """
    result = df.copy()
    for column in df.columns:
        if len(df[column].unique()) == 1:
            result = result.drop(column,axis=1)
    return result

In [ ]:
# load data from the database

def load_data():
    
    engine = create_engine('sqlite:///DisasterResponse.db')
    df = pd.read_sql_table('Message', engine)
    X = df.message.values
    Y = df.iloc[:, 5:]
    Y = drop_constant_columns(Y) # we have to remove constant values in the categories 
    Y = Y.values 
    categories = df.iloc[:, 5:].columns
    categories = categories.tolist()
    
    return X, Y, categories 

X, Y, categories = load_data()


### 2. Write a tokenization function to process your text data

In [ ]:
#– Remove retweet text “RT”
#– Remove hyperlinks
#– Remove hashtags (only the hashtag # and not the word)
#– Remove stop words like a, and, the, is, are, etc.
#– Remove emoticons like :), :D, :(, :-), etc.
#– Remove punctuation like full-stop, comma, exclamation sign, etc.
#– Convert words to Stem/Base words using Porter Stemming Algorithm. E.g. words like ‘working’, ‘works’, and ‘worked’ will be converted to their base/stem word “work”.


def tokenize(text):    
    
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', text)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    
    tweet_tokens = tokenizer.tokenize(tweet)
    
    # removing words that are common
    words = [w for w in tweet_tokens if w not in stopwords.words("english")]
    
    # removing emoticons 
    words = [w for w in words if w not in emoticons]
        
    # removing punctuations 
    words = [ re.sub(r'[^\w\s]', '', w) for w in words]
    
    # removing words that are equal to ''
    words = [ w for w in words if w != '']
    
    # removing words that has numbers 
    words = [w for w in words if w.isalpha() ]

    # removing words that has length equal to 1 
    words = [w for w in words if len(w) > 1 ]
            
    # we apply lemmatization first and then stemming
    # we follow a suggestion from a video that is part of 
    # Data Scientist Nanodegree from Udacity
    words = [WordNetLemmatizer().lemmatize(w) for w in words ]
    words = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words ]
    
    words = [PorterStemmer().stem(w) for w in words ]
    
    
    return words 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
    ('vec', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()), 
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=25, class_weight='balanced', n_jobs=-1)))     
 ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)


In [ ]:

pipeline.fit(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
Y_pred = pipeline.predict(X_test)

In [ ]:
for col in range(Y_pred.shape[1]):
    
    print("Category: {}".format(categories[col]))
    print(classification_report(Y_test[:, col], Y_pred[:, col]))
    

### 6. Improve your model
Use grid search to find better parameters. Below the default parameters are shown.

In [ ]:
# good resource 
# https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

# to get the list of hyperparameters
# pipeline.get_params().keys()

In [ ]:
# Number of trees in random forest
n_estimators = [10 , 25 , 50 , 100]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [2, 5, 8, 10]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [ ]:
parameters = {        
    'clf__estimator__n_estimators': n_estimators,
    'clf__estimator__max_features': max_features,
    'clf__estimator__max_depth': max_depth,
    'clf__estimator__bootstrap': bootstrap            
}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3,  verbose=25, n_jobs=-1)

In [ ]:
cv.fit(X_train, Y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
Y_pred = cv.predict(X_test)

In [ ]:
for col in range(Y_pred.shape[1]):
    
    print("Category: {}".format(categories[col]))
    print(classification_report(Y_test[:, col], Y_pred[:, col]))


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline = Pipeline([
    ('vec', CountVectorizer(tokenizer=tokenize)), 
    ('tfidf', TfidfTransformer()), 
    ('clf', MultiOutputClassifier(SVC(random_state=25, class_weight='balanced')))     
 ])


In [ ]:
gammas = [0.001, 0.01, 0.1, 1, 10, 50 ] # large gammas tend to overfitt 
kernel = ['rbf']

In [ ]:
parameters = {                    
    'clf__estimator__gamma' : gammas, 
    'clf__estimator__kernel' : kernel,    
}


cv = GridSearchCV(estimator=pipeline, param_grid=parameters, cv=3,  verbose=25, n_jobs=-1)

In [ ]:
cv.fit(X_train, Y_train)

In [ ]:
Y_pred = cv.predict(X_test)

In [ ]:
for col in range(Y_pred.shape[1]):
    
    print("Category: {}".format(categories[col]))
    print(classification_report(Y_test[:, col], Y_pred[:, col]))


### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.